In [1]:
import os, glob, sys
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

from ShowColors import *
from ImportUtil import *
%env CITYSCAPES_DATASET = /home/rvygon/data/

/opt/conda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


env: CITYSCAPES_DATASET=/home/rvygon/data/


In [ ]:
#run this cell once
%run  cityscapesscripts/preparation/createTrainIdInstanceImgs
%run  cityscapesscripts/preparation/createTrainIdInstanceImgs

In [2]:
#X_train, y_train = importBatch(1,1500, 0,'train')
x_val, y_val = importBatch(5,200,0,'val')
from keras.utils import to_categorical
#y_train[y_train==255]=19
y_val[y_val==255]=19
#y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [3]:
import numpy as np
def preprocess_pic(X, y):
    y[y==255]=19
    y=to_categorical(y)
    return X, y

def keras_generator(batch_size):
    while True:
        batch_nums = np.random.randint(low=1000, high=2000, size=batch_size)
        batch_input = []
        batch_output = []
        
        for index in batch_nums:
            input_labels, input_images = getData(1, index,type='train')
            input_image = input_images[0]
            input_label = input_labels[0]           
            
            X_img = imread(input_image)
            y_img = imread(input_label)
            
            X_img, y_img = preprocess_pic(X_img, y_img)
            
            batch_input.append(X_img)
            batch_output.append(y_img)
            
        batch_x = np.array(batch_input)
        batch_y = np.array(batch_output)        
        yield batch_x, batch_y
        

In [4]:
#UNET https://github.com/zhixuhao/unet/blob/master/model.py
def unet(pretrained_weights = None,input_size = (1024,2048,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))#drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(8, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(8, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(20, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 5e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [6]:
model = unet()
generator = keras_generator(1)
model_checkpoint = ModelCheckpoint('unet_RV.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(generator,steps_per_epoch=1500,epochs=8,callbacks=[model_checkpoint], validation_data=(x_val,y_val),validation_steps=20)

/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/8
1500/1500 [==============================] - 987s 658ms/step - loss: 6.3590 - acc: 0.3221 - val_loss: 5.8068 - val_acc: 0.3977

Epoch 00001: loss improved from inf to 6.35899, saving model to unet_RV.hdf5
Epoch 2/8
1500/1500 [==============================] - 856s 570ms/step - loss: 6.1983 - acc: 0.3231 - val_loss: 6.0979 - val_acc: 0.3977

Epoch 00002: loss improved from 6.35899 to 6.19827, saving model to unet_RV.hdf5
Epoch 3/8
1500/1500 [==============================] - 854s 570ms/step - loss: 6.1624 - acc: 0.3208 - val_loss: 6.0979 - val_acc: 0.3977

Epoch 00003: loss improved from 6.19827 to 6.16242, saving model to unet_RV.hdf5
Epoch 4/8
1500/1500 [==============================] - 857s 571ms/step - loss: 6.1949 - acc: 0.3252 - val_loss: 6.0974 - val_acc: 0.3977

Epoch 00004: loss did not improve from 6.16242
Epoch 5/8
1500/1500 [==============================] - 856s 571ms/step - loss: 6.1658 - acc: 0.3240 - val_loss: 6.0964 - val_acc: 0.3977

Epoch 00005: loss did no

KeyboardInterrupt: 

In [7]:
new_model = keras.models.load_model('unet_RV.hdf5')
new_model.summary()

results = model.predict(X_train,verbose=1)

colorImage(X_train[0], results[0], 'classes.txt')

AttributeError: module 'keras.backend' has no attribute 'models'

In [ ]:
def eval_model(model):    
        x_val, y_val, filenames = importBatch(1000,0,0,'val')
        x_pred = model.predict(x_val,verbose=1)
        for i in range(x_pred.shape[0]):
            imsave(os.path.join(CITYSCAPES_DATASET,'results', filenames[i]), x[i])
        %run cityscapesscripts/evaluation/evalPixelLevelSemanticLabeling
        
        

    
    